<a href="https://colab.research.google.com/github/salvapineda/bilevel/blob/master/BilevelProgrammingExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PYOMO
!pip install pyomo 
import pyomo.environ as pe
# CBC
!apt-get install -y -qq coinor-cbc
cbc = pe.SolverFactory('cbc', executable='/usr/bin/cbc')
# IPOPT
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
ipopt = pe.SolverFactory('ipopt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 4.8 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 128275 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.11.4+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.108.6+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+rep

In [2]:
# Model
m = pe.ConcreteModel()
# Variables
m.x = pe.Var(within=pe.Reals)
m.y = pe.Var(within=pe.Reals)
m.l1 = pe.Var(within=pe.NonNegativeReals)
m.l2 = pe.Var(within=pe.NonNegativeReals)
m.l3 = pe.Var(within=pe.NonNegativeReals)
m.l4 = pe.Var(within=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = m.y,sense=pe.maximize)
# Constraints
m.c1 = pe.Constraint(expr = m.x >= 2)
m.c2 = pe.Constraint(expr = -m.x-m.y <= -7)
m.c3 = pe.Constraint(expr = m.y <= 9)
m.c4 = pe.Constraint(expr = m.x - 2*m.y <= 1)
m.c5 = pe.Constraint(expr = 6*m.x + m.y <= 45)
m.c6 = pe.Constraint(expr = 1-m.l1+m.l2-2*m.l3+m.l4 == 0)
m.c7 = pe.Constraint(expr = m.l1*(-m.y-m.x+7) == 0)
m.c8 = pe.Constraint(expr = m.l2*(m.y-9) == 0)
m.c9 = pe.Constraint(expr = m.l3*(m.x-2*m.y-1) == 0)
m.c10 = pe.Constraint(expr = m.l4*(6*m.x+m.y-45) == 0)
# Solve problem using NEOS server
ipopt.solve(m).write()
# Print results
print('x =',m.x.value)
print('y =',m.y.value)
print('Optimal value =',m.obj())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 6
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.022221803665161133
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
x = 5

In [3]:
bigM = 100
# Model
m = pe.ConcreteModel()
# Variables
m.x = pe.Var(within=pe.Reals)
m.y = pe.Var(within=pe.Reals)
m.l1 = pe.Var(within=pe.NonNegativeReals)
m.l2 = pe.Var(within=pe.NonNegativeReals)
m.l3 = pe.Var(within=pe.NonNegativeReals)
m.l4 = pe.Var(within=pe.NonNegativeReals)
m.u1 = pe.Var(within=pe.Binary)
m.u2 = pe.Var(within=pe.Binary)
m.u3 = pe.Var(within=pe.Binary)
m.u4 = pe.Var(within=pe.Binary)
# Objective function
m.obj = pe.Objective(expr = m.y,sense=pe.maximize)
# Constraints
m.c1 = pe.Constraint(expr = m.x >= 2)
m.c2 = pe.Constraint(expr = -m.x-m.y <= -7)
m.c3 = pe.Constraint(expr = m.y <= 9)
m.c4 = pe.Constraint(expr = m.x - 2*m.y <= 1)
m.c5 = pe.Constraint(expr = 6*m.x + m.y <= 45)
m.c6 = pe.Constraint(expr = 1-m.l1+m.l2-2*m.l3+m.l4 == 0)
m.c7 = pe.Constraint(expr = m.l1 <= m.u1*bigM)
m.c8 = pe.Constraint(expr = -m.y-m.x+7 >= -(1-m.u1)*bigM)
m.c9 = pe.Constraint(expr = m.l2 <= m.u2*bigM)
m.c10 = pe.Constraint(expr = m.y-9 >= -(1-m.u2)*bigM)
m.c11 = pe.Constraint(expr = m.l3 <= m.u3*bigM)
m.c12 = pe.Constraint(expr = m.x-2*m.y-1 >= -(1-m.u3)*bigM)
m.c13 = pe.Constraint(expr = m.l4 <= m.u4*bigM)
m.c14 = pe.Constraint(expr = 6*m.x+m.y-45 >= -(1-m.u4)*bigM)
# Solve problem using NEOS server
cbc.solve(m).write()
# Print results
print('x =',m.x.value)
print('y =',m.y.value)
print('Optimal value =',m.obj())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5.0
  Upper bound: 5.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 10
  Number of binary variables: 4
  Number of integer variables: 4
  Number of nonzeros: 1
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.02
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
    

In [4]:
# Model
m = pe.ConcreteModel()
# Parameters
m.t = pe.Param(initialize=10**6,mutable=True)
# Variables
m.x = pe.Var(within=pe.Reals)
m.y = pe.Var(within=pe.Reals)
m.l1 = pe.Var(within=pe.NonNegativeReals)
m.l2 = pe.Var(within=pe.NonNegativeReals)
m.l3 = pe.Var(within=pe.NonNegativeReals)
m.l4 = pe.Var(within=pe.NonNegativeReals)
# Objective function
m.obj = pe.Objective(expr = m.y,sense=pe.maximize)
# Constraints
m.c1 = pe.Constraint(expr = m.x >= 2)
m.c2 = pe.Constraint(expr = -m.x-m.y <= -7)
m.c3 = pe.Constraint(expr = m.y <= 9)
m.c4 = pe.Constraint(expr = m.x - 2*m.y <= 1)
m.c5 = pe.Constraint(expr = 6*m.x + m.y <= 45)
m.c6 = pe.Constraint(expr = 1-m.l1+m.l2-2*m.l3+m.l4 == 0)
m.c7 = pe.Constraint(expr = m.l1*(m.y+m.x-7) + m.l2*(-m.y+9) + m.l3*(-m.x+2*m.y+1) + m.l4*(-6*m.x-m.y+45) <= m.t)
# Solve problem using NEOS server
for t in [10**6,10**4,10**2,1,0.1,0.01,0]:
  m.t = t
  ipopt.solve(m).write() 
# Print results
print('x =',m.x.value)
print('y =',m.y.value)
print('Optimal value =',m.obj())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 7
  Number of variables: 6
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.05082535743713379
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
# =====